# 时间窗口加权滑动增强
> @沥川
- 谁能帮忙在时间窗口函数上面改写一下, 简单滑动平均的函数, 加权 * 0.9
- 1M * 24 / 2M * 12 / 3M* 8 / 6M * 4 / 12M*2
- 例如, 最近 2个月的平均, 然后滑动 2 个月窗口, 后面 2个月的平均 * 0.9, 一共取12次

> 分析
- 单月滑动, 联合前月平均 90%
- 双月滑动, 联合前后4个月平均 90%
- ...


In [4]:
import numpy as np
import scipy as sp
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import xgboost as xgb
#import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
train = pd.read_csv('../../raw/LiChuan/trainSaleDate.csv')
test = pd.read_csv('../../raw/LiChuan/test_feature.csv')
submit = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')

## 时间转换

In [6]:
train['sale_date'] = train['year'] * 100 + train['month']
train['sale_date'] = train['sale_date'].astype(int)
train['sale_date'] = pd.to_datetime(train['sale_date'],format='%Y%m',errors = "coerce")
train['sale_date'] = train['sale_date'].dt.to_period('m')

In [7]:
train['sale_date'][0] + 1

Period('2012-02', 'M')

In [8]:
test['sale_date'] = test['sale_date'].astype(int)
test['sale_date'] = pd.to_datetime(test['sale_date'],format='%Y%m',errors = "coerce")
test['sale_date'] = test['sale_date'].dt.to_period('M')

In [9]:
test['sale_date'][0]

Period('2017-11', 'M')

## ZQ: 滑动递减（decay）

In [37]:
def decay_mean_groupby_cid4sale_quantity(train,twin,loop, scale=0.9):
    '''销售额 时间窗口特征 衰减
    例如, 预测 11 月,
        9/10 月销量的均值, 一个特征/
        8/9月销量 * 0.9 均值, 一个特征/
        7/8月销量 * 0.9^2 均值, 一个特征…
        进 1 出 1
    注意:
        滑动次数决定最终输出的数据集尺寸
        进行特征工程时,必须确保所有特征尺寸相同
        否则无法拼接完备
    '''
    _result = pd.DataFrame()
    _step = 0
    if 0 == _step:
        _decay = 1
    else:
        _decay = scale**_step
    for i in range(loop):      
        _sale = train[train['sale_date'].isin(twin)]
        _sale = _sale.loc[:,['class_id','sale_quantity']]
        _sale = _sale.groupby(['class_id']).mean().sale_quantity.reset_index() 
        zoomed = _sale.sale_quantity*_decay
        _sale.sale_quantity = zoomed
        _step +=1
        _sale['sale_date'] = twin[0]-1
        if i == 0:
            _result = _sale
        else:
            _result = pd.concat([_result, _sale])    
        for j,k in enumerate(twin):
            twin[j] = twin[j] - 1
    return _result

In [38]:
# 从最近开始
m = train['sale_date'].iloc[-1]
m

Period('2017-10', 'M')

In [39]:
twin=[m,m-1] #更换窗口
twin

[Period('2017-10', 'M'), Period('2017-09', 'M')]

In [44]:
train_2M_mean_with_decay = decay_mean_groupby_cid4sale_quantity(train,twin, 5)
train_2M_mean_with_decay.rename(columns={'sale_quantity': 'train_2M_mean_with_decay'}, inplace=True)# 替换列名称
train_2M_mean_with_decay.tail()

,class_id,train_2M_mean_with_decay,sale_date
135,961362,46.5,2016-07
136,961962,169.5,2016-07
137,963845,304.0,2016-07
138,973106,0.0,2016-07
139,978089,525.5,2016-07


In [37]:
# 合并
train = pd.merge(train, train_last_2M_mean, on=['class_id','sale_date'], how='left')
train.fillna(0, inplace=True)

'''


'''



## _chaos for debugging...

In [23]:
for k,v in enumerate(twin):
    print(k,v)

0 2017-10
1 2017-09


In [24]:
_sale = train[train['sale_date'].isin(twin)]
_sale = _sale.loc[:,['class_id','sale_quantity']]
_sale = _sale.groupby(['class_id']).mean().sale_quantity.reset_index()
_sale.head()


,class_id,sale_quantity
0,103507,183.0
1,124140,274.0
2,125403,242.0
3,136916,153.5
4,169673,185.5


In [32]:
_sale.sale_quantity.head()

0    183.0
1    274.0
2    242.0
3    153.5
4    185.5
Name: sale_quantity, dtype: float64

In [34]:
_decay = _sale.sale_quantity*0.9

In [35]:
_sale.sale_quantity = _decay

In [36]:
_sale.sale_quantity.head()

0    164.70
1    246.60
2    217.80
3    138.15
4    166.95
Name: sale_quantity, dtype: float64

In [28]:
_result = pd.DataFrame()
_result = pd.concat([_result, _sale])   
_result.head()

,class_id,sale_quantity
0,103507,183.0
1,124140,274.0
2,125403,242.0
3,136916,153.5
4,169673,185.5


In [54]:
#pd_tmp = pd.DataFrame()
_tmp = train[train['sale_date'].isin(window)]
pd_tmp = _tmp.loc[:,['class_id','sale_date','sale_quantity']]
pd_tmp.head()
#train['class_id','sale_date','sale_quantity'].head()


,class_id,sale_date,sale_quantity
9660,125403,2017-10,156.0
9661,136916,2017-10,162.0
9662,178529,2017-10,148.0
9663,194450,2017-10,271.0
9664,198427,2017-10,99.0


In [49]:
pd_tmp.tail()

,class_id,sale_date,sale_quantity
9935,961362,2017-11,0.0
9936,961962,2017-11,0.0
9937,963845,2017-11,0.0
9938,973106,2017-11,0.0
9939,978089,2017-11,0.0


In [55]:
_train_all = train.loc[:,['class_id','sale_date','sale_quantity']]
_train_all.head()

,class_id,sale_date,sale_quantity
0,125403,2012-01,90.0
1,136916,2012-01,665.0
2,178529,2012-01,1739.0
3,194450,2012-01,65.0
4,198427,2012-01,47.0


In [35]:
i = train['sale_date'][0]+1  #从2012-2开始计算
window=[i,i-1] #更换窗口
window

[Period('2012-02', 'M'), Period('2012-01', 'M')]

# 旧嗯哼

~ 发源自 [2.8 窗口函式原型](caijun/caijun0208TimefeatureEngineering.ipynb)~ 皮尔逊（Pearson）相关系数


In [8]:
train.shape,test.shape

((9800, 163), (140, 162))

In [9]:
'''@沥川 处理的时候, 将 train 和 test 合并到一起做, 这样避免处理两次
而且还能避免一些小坑…
'''
train = pd.concat([train, test]).reset_index(drop=True)
train.fillna(0, inplace=True)
train.shape

(9940, 163)

## 窗口函数

In [10]:
def sale_cal_sum(train,window,num):
    traintmp = pd.DataFrame()
    for i in range(num):          
        trainSaletmp = train[train['sale_date'].isin(window)]
        trainSaletmp = trainSaletmp.loc[:,['class_id','sale_quantity']]
        trainSaletmp = trainSaletmp.groupby(['class_id']).agg(sum).sale_quantity.reset_index() #更换所需的函数
        trainSaletmp['sale_date'] = window[0]+1
        if i == 0:
            traintmp = trainSaletmp
        else:
            traintmp = pd.concat([traintmp, trainSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return traintmp

In [11]:
def sale_cal_max(train,window,num):
    traintmp = pd.DataFrame()
    for i in range(num):          
        trainSaletmp = train[train['sale_date'].isin(window)]
        trainSaletmp = trainSaletmp.loc[:,['class_id','sale_quantity']]
        trainSaletmp = trainSaletmp.groupby(['class_id']).agg(max).sale_quantity.reset_index() #更换所需的函数
        trainSaletmp['sale_date'] = window[0]+1
        if i == 0:
            traintmp = trainSaletmp
        else:
            traintmp = pd.concat([traintmp, trainSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return traintmp

In [12]:
def sale_cal_min(train,window,num):
    traintmp = pd.DataFrame()
    for i in range(num):          
        trainSaletmp = train[train['sale_date'].isin(window)]
        trainSaletmp = trainSaletmp.loc[:,['class_id','sale_quantity']]
        trainSaletmp = trainSaletmp.groupby(['class_id']).agg(min).sale_quantity.reset_index() #更换所需的函数
        trainSaletmp['sale_date'] = window[0]+1
        if i == 0:
            traintmp = trainSaletmp
        else:
            traintmp = pd.concat([traintmp, trainSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return traintmp

In [13]:
def sale_cal_mean(train,window,num):
    traintmp = pd.DataFrame()
    for i in range(num):          
        trainSaletmp = train[train['sale_date'].isin(window)]
        trainSaletmp = trainSaletmp.loc[:,['class_id','sale_quantity']]
        trainSaletmp = trainSaletmp.groupby(['class_id']).mean().sale_quantity.reset_index() #更换所需的函数
        trainSaletmp['sale_date'] = window[0]+1
        if i == 0:
            traintmp = trainSaletmp
        else:
            traintmp = pd.concat([traintmp, trainSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return traintmp

In [14]:
def sale_cal_median(train,window,num):
    traintmp = pd.DataFrame()
    for i in range(num):          
        trainSaletmp = train[train['sale_date'].isin(window)]
        trainSaletmp = trainSaletmp.loc[:,['class_id','sale_quantity']]
        trainSaletmp = trainSaletmp.groupby(['class_id']).median().sale_quantity.reset_index() #更换所需的函数
        trainSaletmp['sale_date'] = window[0]+1
        if i == 0:
            traintmp = trainSaletmp
        else:
            traintmp = pd.concat([traintmp, trainSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return traintmp

In [15]:
def sale_cal_std(train,window,num):
    traintmp = pd.DataFrame()
    for i in range(num):          
        trainSaletmp = train[train['sale_date'].isin(window)]
        trainSaletmp = trainSaletmp.loc[:,['class_id','sale_quantity']]
        trainSaletmp = trainSaletmp.groupby(['class_id']).std().sale_quantity.reset_index() #更换所需的函数
        trainSaletmp['sale_date'] = window[0]+1
        if i == 0:
            traintmp = trainSaletmp
        else:
            traintmp = pd.concat([traintmp, trainSaletmp])    
        for j,k in enumerate(window):
            window[j] = window[j] + 1
    return traintmp

## 计算近 3 个月数据

In [16]:
i = train['sale_date'][0]+2  #从2012-4开始计算
window=[i,i-1,i-2]           #更换窗口
window

[Period('2012-03', 'M'), Period('2012-02', 'M'), Period('2012-01', 'M')]

In [17]:
window=[i,i-1,i-2] #每次计算必须重新初始化window
train_last_3M_sum = sale_cal_sum(train,window,68) #计算近3个月的数据总共需要计算67个月
train_last_3M_sum.rename(columns={'sale_quantity': 'sale_quantity_last_3M_sum'}, inplace=True)# 替换列名称

In [18]:
window=[i,i-1,i-2] #每次计算必须重新初始化window
train_last_3M_max = sale_cal_max(train,window,68)
train_last_3M_max.rename(columns={'sale_quantity': 'sale_quantity_last_3M_max'}, inplace=True)# 替换列名称

In [19]:
window=[i,i-1,i-2] #每次计算必须重新初始化window
train_last_3M_min = sale_cal_min(train,window,68)
train_last_3M_min.rename(columns={'sale_quantity': 'sale_quantity_last_3M_min'}, inplace=True)# 替换列名称

In [20]:
window=[i,i-1,i-2] #每次计算必须重新初始化window
train_last_3M_mean = sale_cal_mean(train,window,68)
train_last_3M_mean.rename(columns={'sale_quantity': 'sale_quantity_last_3M_mean'}, inplace=True)# 替换列名称

In [21]:
window=[i,i-1,i-2] #每次计算必须重新初始化window
train_last_3M_std = sale_cal_std(train,window,68)
train_last_3M_std.rename(columns={'sale_quantity': 'sale_quantity_last_3M_std'}, inplace=True)# 替换列名称

In [22]:
window=[i,i-1,i-2] #每次计算必须重新初始化window
train_last_3M_median = sale_cal_median(train,window,68)
train_last_3M_median.rename(columns={'sale_quantity': 'sale_quantity_last_3M_median'}, inplace=True)# 替换列名称

In [23]:
# 合并
train = pd.merge(train, train_last_3M_sum, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_3M_max, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_3M_min, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_3M_mean, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_3M_std, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_3M_median, on=['class_id','sale_date'], how='left')
train.fillna(0, inplace=True)

## 计算滑动窗口为6个月的数据

In [24]:
i = train['sale_date'][0]+5  #从2012-7开始计算
window=[i,i-1,i-2,i-3,i-4,i-5] #更换窗口
window

[Period('2012-06', 'M'),
 Period('2012-05', 'M'),
 Period('2012-04', 'M'),
 Period('2012-03', 'M'),
 Period('2012-02', 'M'),
 Period('2012-01', 'M')]

In [25]:
window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
train_last_6M_sum = sale_cal_sum(train,window,65) #计算近6个月的数据总共需要计算64个月
train_last_6M_sum.rename(columns={'sale_quantity': 'sale_quantity_last_6M_sum'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
train_last_6M_max = sale_cal_max(train,window,65)
train_last_6M_max.rename(columns={'sale_quantity': 'sale_quantity_last_6M_max'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
train_last_6M_min = sale_cal_min(train,window,65)
train_last_6M_min.rename(columns={'sale_quantity': 'sale_quantity_last_6M_min'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
train_last_6M_mean = sale_cal_mean(train,window,65)
train_last_6M_mean.rename(columns={'sale_quantity': 'sale_quantity_last_6M_mean'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
train_last_6M_std = sale_cal_std(train,window,65)
train_last_6M_std.rename(columns={'sale_quantity': 'sale_quantity_last_6M_std'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5] #每次计算必须重新初始化window
train_last_6M_median = sale_cal_median(train,window,65)
train_last_6M_median.rename(columns={'sale_quantity': 'sale_quantity_last_6M_median'}, inplace=True)# 替换列名称

In [26]:
# 合并
train = pd.merge(train, train_last_6M_sum, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_6M_max, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_6M_min, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_6M_mean, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_6M_std, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_6M_median, on=['class_id','sale_date'], how='left')
train.fillna(0, inplace=True)

## 计算滑动窗口为 2 个月的数据

In [27]:
i = train['sale_date'][0]+1  #从2012-2开始计算
window=[i,i-1] #更换窗口
window

[Period('2012-02', 'M'), Period('2012-01', 'M')]

In [28]:
window=[i,i-1] #每次计算必须重新初始化window
train_last_2M_sum = sale_cal_sum(train,window,69) #计算近2个月的数据总共需要计算64个月
train_last_2M_sum.rename(columns={'sale_quantity': 'sale_quantity_last_2M_sum'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
train_last_2M_max = sale_cal_max(train,window,69)
train_last_2M_max.rename(columns={'sale_quantity': 'sale_quantity_last_2M_max'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
train_last_2M_min = sale_cal_min(train,window,69)
train_last_2M_min.rename(columns={'sale_quantity': 'sale_quantity_last_2M_min'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
train_last_2M_mean = sale_cal_mean(train,window,69)
train_last_2M_mean.rename(columns={'sale_quantity': 'sale_quantity_last_2M_mean'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
train_last_2M_std = sale_cal_std(train,window,69)
train_last_2M_std.rename(columns={'sale_quantity': 'sale_quantity_last_2M_std'}, inplace=True)# 替换列名称

window=[i,i-1] #每次计算必须重新初始化window
train_last_2M_median = sale_cal_median(train,window,69)
train_last_2M_median.rename(columns={'sale_quantity': 'sale_quantity_last_2M_median'}, inplace=True)# 替换列名称

In [29]:
# 合并
train = pd.merge(train, train_last_2M_sum, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_2M_max, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_2M_min, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_2M_mean, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_2M_std, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_2M_median, on=['class_id','sale_date'], how='left')
train.fillna(0, inplace=True)

## 计算滑动窗口为 12 个月的数据

In [30]:
i = train['sale_date'][0]+11  #从2013-1开始计算
window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #更换窗口
window

[Period('2012-12', 'M'),
 Period('2012-11', 'M'),
 Period('2012-10', 'M'),
 Period('2012-09', 'M'),
 Period('2012-08', 'M'),
 Period('2012-07', 'M'),
 Period('2012-06', 'M'),
 Period('2012-05', 'M'),
 Period('2012-04', 'M'),
 Period('2012-03', 'M'),
 Period('2012-02', 'M'),
 Period('2012-01', 'M')]

In [31]:
window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
train_last_12M_sum = sale_cal_sum(train,window,59) #计算近12个月的数据总共需要计算58个月
train_last_12M_sum.rename(columns={'sale_quantity': 'sale_quantity_last_12M_sum'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
train_last_12M_max = sale_cal_max(train,window,59)
train_last_12M_max.rename(columns={'sale_quantity': 'sale_quantity_last_12M_max'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
train_last_12M_min = sale_cal_min(train,window,59)
train_last_12M_min.rename(columns={'sale_quantity': 'sale_quantity_last_12M_min'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
train_last_12M_mean = sale_cal_mean(train,window,59)
train_last_12M_mean.rename(columns={'sale_quantity': 'sale_quantity_last_12M_mean'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
train_last_12M_std = sale_cal_std(train,window,59)
train_last_12M_std.rename(columns={'sale_quantity': 'sale_quantity_last_12M_std'}, inplace=True)# 替换列名称

window=[i,i-1,i-2,i-3,i-4,i-5,i-6,i-7,i-8,i-9,i-10,i-11] #每次计算必须重新初始化window
train_last_12M_median = sale_cal_median(train,window,59)
train_last_12M_median.rename(columns={'sale_quantity': 'sale_quantity_last_12M_median'}, inplace=True)# 替换列名称

In [32]:
# 合并
train = pd.merge(train, train_last_12M_sum, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_12M_max, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_12M_min, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_12M_mean, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_12M_std, on=['class_id','sale_date'], how='left')
train = pd.merge(train, train_last_12M_median, on=['class_id','sale_date'], how='left')
train.fillna(0, inplace=True)

In [33]:
train = train.drop(['sale_quantity_this_last_year'], axis=1)

In [34]:
train.to_csv('../../raw/LiChuan/trainallfeatures.csv',index=False, header=True)